In [1]:
import wandb
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback,
)
from datasets import Dataset
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
import pandas as pd
import numpy as np
import torch
import random

/home/uceehuf/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 10:10:15.872904: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 10:10:15.913024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 10:10:15.913055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 10:10:15.914341: E 

In [6]:
! nvidia-smi

Tue Jul 23 13:29:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:25:00.0 Off |                    0 |
| N/A   32C    P0             69W /  300W |    6223MiB /  81920MiB |      6%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch

# Check if the GPU is available
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

0
NVIDIA A100 80GB PCIe


In [4]:

file_path = '../datasets/Train_Data.xlsx'
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# Load the Excel file uing pandas
data_frame = pd.read_excel(file_path,sheet_name=1, engine='openpyxl')

np_frame = np.array(data_frame['Prompt'])
print(data_frame['Prompt'][0])

def generate_rejected(data_frame):
    """
    pre-process the dataset to generate rejected answer from other prompts
    """
    random_seed = 202465
    random.seed(random_seed)
    rejected_T1 = []
    rejected_Amount = []
    rejected_T2 = []
    # rejected_reason = []

    T2_labels = ['Fully Reimbursable','Partially Reimbursable','Not Reimbursable','Further Clarification Required']

    for i in range(0,len(data_frame)):
        all_change_made = False
        while not all_change_made:
            # T1 toggle:
            if data_frame['Classification T1'][i] == 'Policy Not Violated':
                T1 = 'Policy Violated'
            else:
                T1 = 'Policy Not Violated'
            # Amount ramdom number:
            Am = round(random.uniform(0, 1000),2)

            # T2 randomly choose a new label from the remaining labels
            remaining_labels = [label for label in T2_labels if label != data_frame['Classification T2'][i]]
    
            T2 = random.choice(remaining_labels) 

            if (T1 != data_frame['Classification T1'][i]) and (Am != data_frame['Amount'][i]) and (T2 != data_frame['Classification T2'][i]):
                all_change_made = True


        # Following was codes for only 1 field changed
        # change_made = False
        # counter = 0
        # while not change_made:
            
        #     binary_change = bin(random.randint(1, 15))[2:].zfill(4)
        #     counter += 1
        #     rand_index = random.randint(0, len(data_frame)-1)
        #     if binary_change[0] == '1':
        #         T1 = data_frame['Classification T1'][rand_index]
        #     else:
        #         T1 = data_frame['Classification T1'][i]
        #     if binary_change[1] == '1':            
        #         Am = data_frame['Amount'][rand_index]
        #     else:
        #         Am = data_frame['Amount'][i]
        #     if binary_change[2] == '1':
        #         T2 = data_frame['Classification T2'][rand_index]
        #     else:
        #         T2 = data_frame['Classification T2'][i]
        #     # if binary_change[3] == '1':
        #     #     Re = data_frame['Reasons'][rand_index]...

        #     if (T1 != data_frame['Classification T1'][i]) or (Am != data_frame['Amount'][i]) or (T2 != data_frame['Classification T2'][i]):
        #         change_made = True
        #     elif counter > 10:
        #         print(f"generate failed for prompt {i}")

        # now that changed are made:
        rejected_T1.append(T1)
        rejected_Amount.append(Am)
        rejected_T2.append(T2)
        # rejected_Re.append(Re)

    rejected_df = pd.DataFrame({
        'rejected_T1': rejected_T1,
        'rejected_Amount': rejected_Amount,
        'rejected_T2': rejected_T2,
        # 'rejected_reasons': rejected_Re,
    })

    # Concatenate the original DataFrame with the rejected DataFrame
    data_combine = pd.concat([data_frame, rejected_df], axis=1)
    return data_combine

temp1 = generate_rejected(data_frame)


For a fortnight, I relied on a TFL travel card for my daily commute from home to the GNEI campus, totaling £50 in expenses. What's the reimbursement amount?


In [5]:
temp = temp1[temp1["Category"] == "Accomodation and Sustenance"]
# Convert the pandas DataFrame to a Hugging Face Dataset
tr_data = Dataset.from_pandas(temp)

print(tr_data)


Dataset({
    features: ['Prompt', 'Category', 'Classification T1', 'Classification T2', 'Amount', 'Test ID', 'rejected_T1', 'rejected_Amount', 'rejected_T2', '__index_level_0__'],
    num_rows: 200
})


In [6]:
from transformers import DataCollatorWithPadding, AutoTokenizer, Trainer, TrainingArguments
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

torch_dtype = torch.float16
# Load tokenizer and model
# Mistral 7B it
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
    trust_remote_code=True, device_map="auto", torch_dtype=torch_dtype)


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


In [7]:
context_full = """

GNEI Expenses Policy
Summary Guidance
Subject
Page No.
Train Travel
This should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel via GNEI's approved travel provider.


Taxis
Taxis may be used where there is a clear requirement or where they are the most economic practical means of transport.


Business Mileage
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter, Motorcycles - 26p per mile, Bicycles - 22p per mile.


Air Travel
This should be booked using GNEI’s preferred travel provider where possible. For flights of five hours or less, staff should book restricted tickets in economy. For flights lasting five hours or longer staff may travel premium economy. Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class.


Hotel Rates
UK: should not exceed £210 including VAT (room only) in London or £140 including VAT (room only) outside of London. The cost of accommodation at or near the normal place of work wouldn’t qualify as business travel and would therefore be subject to tax.
Overseas: not subject to a fixed rate or limit. Claimants should instead look to apply this policy's general principles (section 3) when assessing reasonableness and use either HMRC's overseas benchmark rates and / or this policy's equivalent UK rates for guidance


Meals whilst away overnight
UK Breakfast: maximum of £12 receipted including VAT and service; unreceipted £5.5
UK Lunch: maximum of £18 receipted including VAT and service; unreceipted £6
UK Dinner: maximum of £36 receipted including VAT and service, unreceipted £17.
UK 24-hour rate: In line with the above, up to £66 receipted (£28 unreceipted).
Overseas: not subject to a fixed rate or limit. Claimants should instead look to apply this policy's general principles (section 3) when assessing reasonableness and use either HMRC's overseas benchmark rates and / or this policy's equivalent UK rates for guidance.


Line Rental
Line rental cannot be claimed; only the cost of business calls on a call-by-call basis can be claimed.


Broadband
Home internet connection cannot be claimed.


Staff and Student Entertaining
This means food or drink for two or more members of staff or registered GNEI students in connection with GNEI business activities. The cost of entertaining should not exceed £22 per head.


Staff Social Functions
Events that are not in connection to GNEI business activities e.g. Christmas lunches/parties, end of term socials, retirement parties etc. The GNEI department contribution should be no more than £22 per head.


Business entertaining
Cost should be appropriate and not exceed £44 per head including alcohol and service unless exceptional circumstances apply.


Professional Subscriptions
GNEI will only reimburse staff, or pay on their behalf, annual subscriptions or memberships to a professional body where either:
Savings to GNEI arising from membership, for example reduced conference attendance fees, exceed the cost of membership, or
Membership is mandatory in order to be able to teach on a professionally accredited course.
In all cases, the professional body must feature in the list of approved organisations published by HMRC.





1. Introduction 
This Expenses Policy (‘the Policy’) provides guidance to all individuals claiming reimbursement of reasonable expenses incurred in connection with GNEI business.
GNEI is a charity and a large recipient of public funds from UKRI and the Office for Students, grants from other public bodies, charities and fees paid by students. All expenditure should be appropriate and modest in scale.
This policy has been prepared in accordance with Income Tax and National Insurance Contribution regulations and Her Majesty’s Revenue and Customs (HMRC) regulations.



2. Scope
This policy applies to all spend on GNEI activities including from research grants and discretionary accounts as well as departmental codes. 
This policy covers the following areas of expenditure:
Travel
Overnight costs and allowances
Telephone and internet costs
Entertainment & Hospitality
Training
Subscriptions
Advances
Other Expenditure
For items that do not fall under any of the above headings, please refer to your local finance team or Accounts Payable at expenses@gnei.ac.uk.
For expense claim procedures, please refer to Annex 1.
Generally, GNEI will not reimburse individuals for:
Broadband/Internet Provision
Equipment
Fines
Gifts
Insurance
Mobile Phone Contracts/Hardware
Personal Expenditure
Professional Subscriptions
Stationery
For further details on the above exclusions, please refer to Annex 2.
Information on VAT aspects which are common to all claims can be found at Annex 3.
This policy also provides information on subsistence payments for visiting Research Fellows, please refer to Annex 4.
Childcare and caring responsibility can be claimed in certain circumstances, please refer to Annex 5.



3. General Principles
GNEI employees and all others engaged in GNEI activity (collectively claimants) will be reimbursed for the actual cost of expenses incurred, wholly, exclusively and necessarily in the performance of GNEI activity as prescribed in this policy.
The expense must be justifiable and reasonable according to the information in this policy, and the claim should always be prepared honestly, legally and responsibly. Any breach of the policy could lead to disciplinary action, up to and including dismissal.
Whilst this policy aims to provide comprehensive guidance on reimbursable out-of- pocket expenditure, it is recommended that staff still seek advance approval from their budget holder or department manager in any situation where interpretation of this policy is in doubt.
Employees should be able to procure the majority of goods and services required for GNEI business using the standard purchase to payment procedures using the GNEIFinance system.
It is the responsibility of authorised signatories to ensure that this expenses policy is upheld; and it should be noted that individual claims falling outside of the policy may be subject to review and/or rejection by Finance.
Electronic images of original receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used should include details of what goods or services have been purchased. Original paper receipts do not need to be stored locally, unless they relate to sponsored research and it is explicit in the terms of the grant that original paper receipts need to be retained.
When making a claim for a group, the most senior person present should pay for the expenditure and make the claim.
Expenses should be submitted as soon as possible after they have been incurred. Claims must be made within three months. Approval should be gained from the relevant School or PS Head of Finance if the claim is over three months old.
It costs a fixed amount to process any claim, so claimants should endeavour not to submit claims for less than £33, unless they represent the total of expenses in a three-month period.
If a claimant inadvertently makes an error with an expense claim and has been reimbursed, GNEI will recover the amount from them.
Expenses must not be used as a way of rewarding people or encouraging them to work in remote locations.
GNEI will not meet the cost of expenses for spouses, partners, other family members or friends of GNEI staff.
Claims are checked by Accounts Payable staff and any deemed to be fraudulent will be investigated and referred to GNEI’s Internal Auditors. If a claimant is found to have submitted a fraudulent claim they will face disciplinary proceedings, leading to penalties up to and including summary dismissal.



4. Travel
This section details what can be claimed as travel expenses and provides information on specific modes of transport and other expenses associated with travel.

General guidance for travelling on GNEI business
The cost of business travel, that is, journeys away from your normal place of work while undertaking GNEI business can be claimed.
In line with the GNEI travel policy, all business trips in the UK or abroad must be authorised in writing in advance by the GNEI budget holder responsible for making the funds available. It is recommended that this is attached to the claim.
The budget holder / manager must agree that the travel is necessary and that alternative methods such as conference call, video conference, phone or email cannot be used or are not appropriate. Consideration should also be given as to how to minimise the environmental impact of travel.
This authorisation ensures that anticipated costs are in line with departmental plans, are eligible under the terms of any associated funding, and that subsequent expense claims can be submitted with confidence, subject to the provisions of this policy.
GNEI has preferred travel suppliers for airline, hotel and car hire bookings details on these suppliers can be found on the Procurement website. These suppliers should be used for travel bookings where possible.

Private travel costs
The following cannot be claimed:
Travel between home and normal place of work.
Business travel broadly similar to the claimant’s normal commute.
Recreational travel and accommodation at or near the business travel destination.
Travel or accommodation for family or friends accompanying the claimant on the business journey.

Rate of exchange
If a credit card was used to pay for overseas expenses, the rate charged by the credit card issuer can be claimed. A copy of the statement should be used to evidence the rate used.
For employees GNEIExpenses will use a default rate for translating foreign currency transactions; this can be overridden if different to the rate at the time of expenditure and fluctuations will be allowed within a set tolerance. If the automatic rate is overridden, a copy of the bank statement should be included with the claim to back up the rate used.
Buying foreign currency is not a genuine business expense and cannot be claimed.

Travel Insurance
Travel Insurance needs to be activated before traveling overseas, travelling in the UK for fieldtrips or travelling on a business trip where there is an overnight stay or air travel involved.
Insurance can be activated by completing a travel insurance request form, which can be found in the Insurance Section on the Finance website.
The Insurance team will issue a cover note and provide details of the policy. There is no cost to departments for travel insurance. For regular travellers the Insurance Section can issue annual cover notes on request.
If a claimant has an existing medical condition and is not travelling against their doctors’ orders, this is covered by the GNEI Travel Insurance Policy at no extra cost.
Personal travel insurance cannot be claimed as GNEI already provides cover for business travel, please refer to Annex 2 for further information on this exclusion.

Public Transport
Journeys made on rail, bus, ferry, river boat, underground, metro or tram services – can be claimed.
The most economical method of travel should be used. For example, if the claimant uses public transport in London regularly, they should use an Oyster card rather than paying for single journeys by cash.

Train journeys
These should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel.
If the claimant does travel first class, an explanation must be included on the claim and if necessary the claim will be passed to the relevant School or PS Head of Finance for approval.
Trains should be booked in advance where possible via GNEI's approved travel provider or similar rail travel provider as this is the most cost effective option. Tickets can then be collected at the relevant station or sent via post.

Tube and bus journeys in London
If a London Transport Oyster or contactless credit/debit card is used for business journeys, the details of each individual journey should be entered on the claim, being specific about locations, date and times and purpose of journey.
Only the relevant journey should be claimed, not the whole top-up cost, unless the whole top-up will be used for GNEI business travel. Receipts are not necessary for individual journeys but are required if claiming for the whole top- up fee.

Receipts
Relevant tickets or documentation should be kept and attached to the expense claim electronically.
Claimants who have a rail or Oyster season ticket enabled to use the underground can only claim for travel if extra expenses are incurred that are not covered by the season ticket and can provide a receipt as proof of the expense.
Ticket machines will provide receipts if the ticket is retained by the automatic barrier at the end of a journey.

Taxis
Taxi fares can only be claimed for journeys where:
It is clearly the most efficient and cost-effective method of transport.
Alternative methods of transport are impractical due to pregnancy, disability, illness or injury, luggage or similar.
A member of staff is working in the office very late, i.e. after 11pm.
Prior approval has been given, where feasible, from the budget holder or departmental administrator/manager.

Business Mileage
If a car, motorcycle or bicycle has been used on a business trip the costs can be claimed using a mileage allowance – that is, a sum of money for every business mile travelled. GNEI does not reimburse claims based on actual petrol receipts.
GNEI has standard rates for these allowances, depending on the circumstances surrounding the trip (see below).
Only additional costs incurred, above what would normally be spent for commuting to work (which are not eligible as a business expense) can be claimed.
A car should only be used if it is cheaper and more efficient than public transport. Cars should be shared with other staff members wherever possible.
The claim should state the start and end points of the journey, along with the number of miles being claimed.
Mileage is paid at the HMRC approved mileage allowance rates:
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter.
Motorcycles - 26p per mile.
Bicycles - 22p per mile.

Car/bicycle/motorcycle insurance
It is important that adequate personal insurance cover for business use is in place, as cars, bicycles or motorcycles belonging to claimants are not covered by GNEI’s insurance. Personal insurance cannot be claimed, see Annex 2 for further information.
If a staff member uses or allows another member of staff to use a vehicle on GNEI’s business that is insured for third party risks only, GNEI will not be held liable for any of the following:
damage or repairs to the vehicle
property lost or damaged
personal injuries sustained or caused during or as a result of such use.

Car sharing
Only the driver can claim mileage for a journey where two or more GNEI staff share a car. The following cannot be claimed:
Insurance cover for private cars.
Accident or breakdown recovery costs, such as AA membership.

Air Travel
Air travel should be booked using GNEI’s preferred travel provider where possible.
GNEI prefers lower fares, based on flight arrival and departure times which cannot be changed. Where there is a need for flexibility in departure or arrival times, a different ticket option will have to be requested. Flexible tickets for flights under three hours require approval by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.
Membership of a frequent traveller scheme and potential air miles benefits must not deflect from using the most cost-effective airline option for travel.
GNEI does not meet the travelling costs of:
Employees of contractors
Spouses, partners, other family members or friends.
Classes of travel:
For flights of five hours or less, staff should book restricted tickets in economy.
For flights lasting five hours or longer staff may travel premium economy.
Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class. This must be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Toll charges
Toll charges paid for in the course of a business journey can be claimed. Toll bridges or roads rarely give receipts; therefore the name of the toll bridge or road should be included on the expense claim.

Congestion charges
GNEI will not reimburse the cost of any congestion charge for staff or visitors except in exceptional circumstances, e.g. where a member of staff has to collect/deliver equipment to a location and the journey could not be completed on public transport. This must be approved in advance by the relevant School/PS Head of Finance.

Car Parking
Reasonable parking costs on business visits and journeys away from the office can be claimed.
Hotel parking costs can be claimed if they are charged separately on the bill.
If a claimant has to drive to their place of work as part of a business journey, e.g. to collect equipment, parking costs can be claimed. Details of the reason should be included with the claim.
The cost of parking at an entertainment venue cannot be claimed.
VAT can be reclaimed (where it's charged) on parking fees which cost less than £28 even if the claimant is not able to get a VAT receipt.

Car Hire
Approval must be obtained from the budget holder or departmental administrator/manager before a car is hired.
If hiring a car is the cheapest mode of travel, it should be booked through GNEI's approved car hire provider (for UK or overseas).
The car should only be used for business purposes.
Rates are inclusive of CDW (Collision Damage Waiver), TPI (Third Party Insurance) and PAI (Personal Accident Insurance).
Any petrol bought solely for business purposes can be claimed. A VAT receipt should be obtained for purchases, a credit card receipt is not sufficient.

Group Travel
This is in relation to a staff conference or student field trip. Prior approval from the budget holder or departmental administrator/manager should be obtained before booking a group trip.
The same rules apply for each category of travel and overnight costs as for travelling as an individual.
All travel (including coach hire) and overnight costs should be booked in advance.
GNEI's approved travel provider provides a group travel & conference booking facility, where all elements of the trip can be arranged. This can be used for arranging a business conference, as well as student field trips.
Please ensure adequate insurance is in place for all staff and students on the trip. Travel insurance needs to be activated prior to the trip by using the Insurance section of the Finance website.

Travel Incidentals
Claimants can claim the following goods or services if they are related to the business purpose of the trip:
The cost of obtaining a visa for working abroad on GNEI business. These can be obtained from GNEI's approved travel provider.
The cost of any vaccinations needed for working abroad.
There may be occasions when a claimant needs a second passport, for example when travel to one country could lead to immigration difficulties in travelling on business to some other countries. GNEI will pay this expense.
Approval must be obtained in advance from the budget holder or departmental administrator/manager.



5. Overnight costs and allowances
Such costs are subject to the requirement for pre-approval in section 4.3 above.

Hotels
In line with the GNEI travel policy, Hotels must be booked via GNEI's approved travel provider for both overseas and UK travel, with recovery through expenses only permitted in those cases set out in Section 3.1 of that policy and subject to an appropriate level of advance approval as set out in section 3.2. The approval obtained must be attached to submitted expense claims alongside receipts for the room cost, breakfast, dinner and other eligible incidentals.
Hotel city tax: from time to time, additional local city tax charges may apply for accommodation. These can be reclaimed via expenses.

UK Hotels
As stipulated by HMRC, the cost of accommodation at or near the normal place of work does not qualify as business travel; any claim would be subject to tax on the individual as a reimbursement of personal expenditure. Such costs may not be claimed, and staff are asked to bear this in mind when making their travel arrangements.
The cost of a hotel in the UK should not exceed £210 including VAT (room only) in London or £140 including VAT (room only) outside of London. If a hotel cost is higher than these limits, this should be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Overseas hotels
The cost of overseas accommodation may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this in mind, the University does not look to prescribe fixed limits for overseas accommodation but instead provide guidance on what a reasonable claim would be so that claimants and approvers are able to make a fully informed assessment.
In the first instance, employees should expect to be reimbursed for hotels that are equivalent to the most appropriate GBP rate for UK travel detailed in section 5.5. For overseas stays the expectation of the University is that an employee should book accommodation that is equivalent to a UK 3-star level. For areas where there is deprivation or a high security risk then the rating may be increased to 4-star.
Employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit nor an automatic entitlement but should be considered a point of reference only.
Employees should be aware however that claims materially in excess of the HMRC rates are likely to be subject to challenge either during the approval process or before settlement by the central processing teams. Where this is expected to be the case, claimants should ensure that clear justification and confirmation of pre-approval from the relevant budget holder is attached to the claim as per Section 4.3.
Where attendance at an international conference requires specific accommodation (for example by virtue of a fixed attendance package or as a necessary means of realising the benefits of the conference), this factor should also be documented with pre-approval attached to the claim.
Incidentals
All necessary incidentals (including brief personal calls home) should be receipted and claimed:
Newspapers, bar drinks, mini bar, hotel video and health and fitness facilities cannot be claimed.
Laundry costs can only be claimed if absolutely necessary, the cost is reasonable and appropriate and the stay away is for at least 5 consecutive nights.

Telephone, Fax and Internet Charges
The cost of business calls, fax and internet access charges for business use, as long as they are included on an itemised bill, can be claimed.
When working overseas a telephone card can be purchased as this reduces the cost of local calls, the cost of this instead of calls from hotels can be claimed.

Meals
When travelling on GNEI business, employees are entitled to recover the cost of reasonable subsistence, namely breakfast, lunch, and dinner. Meals cannot be claimed if already included in hotel accommodation rates being claimed for.
If several members of staff have a meal together, the most senior person should pay and submit the expense claim. In such a case no-one other than the most senior person should claim an allowance for the meals. Please ensure that it is made clear in the justification field of GNEIExpenses that the receipt is covering more than one employee’s meal, and each employee name (covered by the receipt) should be listed.
The following rates include the cost of alcoholic beverages.

UK Meals
Breakfast. If breakfast is not included in the hotel room rate, the cost can be claimed, which should be receipted and claimed as part of the overnight stay to a maximum of £12 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for breakfast.
Lunch. If the claimant is working away from their normal place of work, lunch can be claimed to a maximum of £18 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for lunch.
Evening meal. The cost of an evening meal can be claimed which should be receipted and claimed as part of the overnight stay to a maximum of £36 including VAT and service. If it is not possible to obtain a receipt, then £17 may be claimed for dinner.
24-hour rate: Where UK travel includes an overnight stay and incorporates breakfast, lunch and dinner a single claim of up to £66 receipted (£28 un-receipted) may be made. In these cases, individual meal rates need not be applied, and the claim will be assessed on an aggregate basis in order to allow for greater flexibility. This 24-hour rate will not apply where any one of the meals is included within a room rate, and claimants must still provide receipts for each separate meal. Only actual spend will be reimbursed and all claims remain subject to the overriding principles of reasonableness and value for money.

Overseas Meals
The cost of meals may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this mind, the University does not look to prescribe fixed limits for overseas meals but instead provide guidance on what a reasonable claim would be.
In the first instance, employees should expect to be reimbursed for meals that are equivalent to the standard of meal that can be purchased in the UK within the set GBP rates for UK travel detailed in sections 5.18 to 21.
As an additional location-specific guide, employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit for claims (noting that they’re updated infrequently) but nor do they represent an automatic entitlement. The rates should be considered a point of reference only and all claims must uphold the general principles set out above.
In line with section 3 above, employees will only be reimbursed for actual costs incurred as evidenced by original receipts. Where receipts for overseas subsistence cannot be obtained, employees may claim at the unreceipted GBP rates set out in sections 5.18 to 5.21 up to a maximum of £28 per day for three meals.

Rented Accommodation (This section applies to GNEI staff and students only, the policy for visiting Research Fellows is detailed in Annex 4).
Long-term assignments
If a member of staff is working away from home on a longer-term assignment – usually for six months or more – the budget holder may require them to rent a property rather than stay in a hotel.
The lease should be taken out in the employee’s name as they will be the sole occupant of the property; prior approval from budget holder or departmental administrator/manager must be obtained before the lease is agreed.
The monthly rent, council tax, service charges, TV licence and utility bills (including telephone line rental) including VAT can be claimed. Alternatively, GNEI can pay the rent directly on invoice via Accounts Payable.
If several colleagues are working away from home working on the same project and sharing a property, only one of the staff should claim expenses for the cost of the accommodation and associated bills.

Relocation Expenses
Please refer to the Relocation Expenses Guide for further details.



6. Telephone and Internet costs
This category details what can be claimed when using a telephone for business purposes.
Always use the cheaper of either mobile or land line.

GNEI mobile phone users
As calls and line rental are paid directly by GNEI, no claims for business calls, from either a personal mobile or a home landline should be made by staff in receipt of a GNEI mobile phone.
If it is decided that an employee requires a mobile phone for work purposes, it should be obtained via ISD’s Managed Mobile service.

Own mobile phone users
Line rental cannot be claimed. Only the cost of business calls on a call-by-call basis can be claimed, and an itemised VAT bill with the relevant calls highlighted must be provided as a receipt and attached to the claim.

Home personal land line
Only the cost of business calls on a call-by-call basis can be claimed, and an itemised VAT bill with the relevant calls highlighted must be provided as a receipt and attached to the claim.
Any telephone line rental or standing charges cannot be claimed.

Wi-Fi
Short Wi-Fi internet session fees e.g. at airports and hotels while on GNEI business can be claimed, a receipt must be provided.

Broadband/Internet Provision
Home internet connection cannot be claimed for in any circumstances.



7. Entertainment and Hospitality
This section details what can be claimed when entertaining staff, students or external visitors – that is, providing food, drink or other hospitality.
The nature of the event should be established:
Staff and student entertaining: the event is to primarily entertain GNEI staff and students with some (or no) visitors.
Business entertaining: the event is to primarily entertain visitors with some GNEI staff and students attending.
A record of all staff and student attendees and their GNEI department names must be included with any claim – this is an HMRC requirement.
All expenditure should be reasonable and appropriate.
The cost of entertainment and / or hospitality should not be paid for using departmental purchasing cards under any circumstances.

Hospitality for meetings
Food can only be provided when it is necessary to hold a meeting during a normal mealtime (breakfast - before 9.00am, lunchtime - between 12.00 and 2.00pm, or evening - after 6.00pm), and it is not practical to expect staff to bring their own food. No alcohol can be provided.
Where possible the meeting should be held on GNEI premises and food provided by GNEI's catering providers.

Staff and student entertaining/hospitality
This means food or drink for two or more members of staff or registered GNEI students in connection to GNEI business activities. It includes working lunches/dinners and team- building events. Approval from the budget holder or departmental administrator/manager must be sought before organising or paying for any staff or student entertaining.
The cost of entertaining should not exceed £22 per head.

Staff social functions
These are events that not in connection to GNEI business activities e.g. Christmas lunches/parties, end of term socials, retirement parties etc.
In general, staff should be expected to contribute to the cost of these events. The GNEI department contribution should be no more than £22 per head.
Any expenditure on social functions should be approved in advance with the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance.
Costs for major staff entertaining functions (for example annual social) should be invoiced directly to GNEI, not entered on individuals' expense claims.

Business entertaining
The most senior person present should pay the bill for any business entertaining and submit the claim.
Hospitality, where feasible, should be pre-approved by the budget holder or departmental administrator/manager.
The cost should be appropriate and not exceed £44 per head including alcohol and service.
If for an exceptional reason the costs exceeds £44 per head, including alcohol and service, this should be approved by the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance.
A record of all attendees and their organisation names must be kept and included with the claim; this includes any GNEI staff or students attending.
GNEI staff or students attending a business entertaining event in a support role, e.g. greeting guests, without participating in the hospitality should not be included in the numbers attending. Treat any costs arising from their attendance as part of the cost of the function and claim them as such. 



8. Training
Training and study costs can only be claimed if the training is for a business purpose.

External training courses
External training courses should be booked using a GNEI Purchase Order and paid directly by GNEI on invoice via Accounts Payable.
Prior approval for the expense from the budget holder or departmental administrator/manager must be obtained before booking the course.

Study Assistance
The GNEI Study Assistance Scheme (SAS) provides a range of support for GNEI staff seeking to gain qualifications to support their work and careers.



9. Other Expenditure
Professional body subscriptions
GNEI will only reimburse staff, or pay on their behalf, annual subscriptions or memberships to a professional body where either there are:
Savings to GNEI arising from membership, for example reduced conference attendance fees, exceed the cost of membership, or
Membership is mandatory in order to be able to teach on a professionally accredited course.
In all cases, the professional body must feature in the list of approved organisations published by HMRC.

Books, journals and magazines/periodicals
Most books and journals should be purchased via a GNEI PO and paid directly by GNEI on invoice via Accounts Payable.
Books should only be purchased directly by a member of staff when this is not possible, e.g. Amazon does not accept purchase orders.

UK Work Permit (including renewal)
In exceptional circumstances, and with prior approval from the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance, the cost of obtaining or renewing a visa for working in the UK may be claimed through expenses.

Specialist clothing
If a job requires specialist clothing, this should usually be purchased via a GNEI PO and paid directly by GNEI on an invoice via Accounts Payable.

Subject Fees
Where individuals are paid cash to take part in a research activity/survey, the individual is required to sign a form stating that they have received the cash from the claimant. This form should be attached to the expenses claim form as a receipt.



10. Advances
Foreign Advances
These are available for overseas travel on behalf of GNEI where other means of paying for the trip are not available.
Staff may apply for foreign currency, traveller’s cheques or for the sterling equivalent to be paid directly into their bank account. Please go to the Foreign Advances Policy for further information.

Research Advances for Subject Fees
Staff may apply through their departments for a Research Advance for the purpose of remunerating Subjects for taking part in experiments, to a maximum of £55 cash per subject for the duration of the experiment.
Subjects may claim in addition to this their travel (where allowed by the grant) from the advance on the production of the original receipt.
If the Subject is to be paid more than £55 (not including travel) or they will be taking part in multiple experiments within the same research where the total is more than £55, this should be paid to the Subject directly by GNEI using a general expense claim form and not through the advance (due to HMRC tax regulations). Please go to the Research Advances Policy for further information.




Annex 1 – Expense claim procedures
How to make a claim
All GNEI employees and students with a UK bank account must claim expenses online via the GNEIExpenses module on GNEIFinance. Anyone else should use the external visitor system.

Receipts
For GNEI employees, electronic images of original VAT receipts must be attached to all claims. Once submitted each claim will be routed automatically to the appropriate approver(s) and Accounts Payable.

For students or external individuals, a claim will be processed once the approved claim form and all supporting paper receipts have reached Accounts Payable.

Please see Annex 3 for further information on VAT receipts. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used must include details of the goods or services purchased. 

If the necessary receipts or documentation is not available claimants should obtain approval from the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance prior to submitting the claim, and provide evidence of this approval attached to the claim. Use the Justification field in GNEIExpenses to notify Accounts Payable that the approval is attached.

Approval
GNEI operates a system of delegated authorisation to staff and approval limits are set out in the GNEI Financial Regulations. Expense approvers should be aware of the funds available in the project to which the claim is being charged to.

All expense claims require authorisation prior to payment. This authorisation is in line with GNEI’s Financial Authorised Signatory hierarchy. It is the responsibility of the claimant to ensure that the approver will confirm authorisation for each item in the claim. Claimants may be personally liable for expenses not subsequently approved.

Approver(s) must personally review and then approve the claim. Approvers are expected to review claims promptly and consider the following before approving:
The appropriateness and quantum of the expenditure
The expenses category chosen for each item
The project code(s) charged
Any policy violations
That receipts or receipt images correspond to the claim and constitute itemised receipts, not credit card slips/bank statements.
That items within the claim meet the terms of the research funder’s terms and
conditions (if related to sponsored research).

If there are queries on any of the above, claims should be challenged by the Approver and
rejected if incorrect.

Expense claims not appropriately authorised will be returned.




Annex 2 – Exclusions
In exceptional circumstances claims for some of these items may be allowed, prior approval should be obtained from the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance. If prior approval is not obtained there is no guarantee that expenditure will be reimbursed.

Broadband/Internet Provision
Home internet connection cannot be claimed for in any circumstances.

Entertainment and Hospitality
GNEI will not pay for:
Clothing for entertaining e.g. dress or suit hire
Overnight accommodation

Equipment
All equipment should be purchased directly by GNEI via a Purchase Order (PO) and paid directly on invoice via Accounts Payable. Categories of equipment include, although not exclusively, IT hardware & software, laboratory equipment & consumables. GNEI have negotiated agreements with Contracted Suppliers for equipment.

Fines
GNEI does not pay:
Fines or fixed penalties
Administration fees charged by third parties for recovering fines
Fines for motoring offences

Gifts
GNEI will not pay for gifts in any circumstances. Gifts for staff should be purchased using personal funds or via a staff collection. Gifts should not be claimed via expenses, nor purchased via iProcurement or a GNEI Purchasing Card.

Insurance
Travel insurance should be obtained from the Insurance section of the GNEI Finance website. Personal travel insurance policies will not cover claimants whilst abroad on GNEI’s behalf, and cannot be claimed as GNEI already provides cover for business travel. If a claim is received for travel insurance it will not be paid. Personal car/bicycle/motorcycle insurance cannot be claimed.

Mobile Phone Contracts/Hardware
Monthly contract charges and hardware cannot be claimed for. If a mobile phone is required for work purposes it should be obtained from ISD Telecoms. 
GNEI will pay for the cost of business calls made from a personal mobile phone, please see the ‘Telephone costs’ section of the GNEI Expenses Policy.

Personal Expenditure
Personal incidental costs while away on GNEI activity cannot be claimed. These include, although not exclusively, newspapers, bar drinks, mini bar, hotel video and health and fitness facilities.

Stationery
All stationery must be purchased via GNEI's approved supplier, via a PO and paid directly on invoice via Accounts Payable.




Annex 3 – VAT
VAT
GNEI can reclaim the VAT charged on some business expenses. In order to make a proper claim GNEI needs valid receipts to provide appropriate evidence. It is the claimant’s responsibility to obtain a receipt showing VAT where it applies.

As defined by HMRC, original VAT receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of expenditure. All receipts should show the following information:
Name, address and VAT number of the supplier
Date of supply and date of issue if different
Description of the goods or services
For each VAT rate the total amount payable including VAT and the VAT rate charged
Name and address of person to whom the services are supplied (if appropriate)
Unit price (if the supply cannot be broken down into countable elements then the total tax exclusive price will be the unit price)
The total amount of VAT charged



Annex 4 - Visiting Research Fellows Subsistence Payments
Subsistence payments should be claimed via a Visiting Research Fellow form, which will be processed as a manual  expense form if the following conditions are met:
The general rule is to determine if an overseas visitor is an employee or not. If the overseas visitor does not undertake any duties (other than research) normally performed by GNEI staff and they are not subject to the usual terms and conditions of employment, then payments may be made to them without deduction of PAYE and NI.
Usual terms and conditions of employment would include such matters as the ability of GNEI to determine hours worked, location of work, to set a probationary period and entitlement of the payee to holiday pay, sick pay and to join a pension scheme.

The individual must be visiting for less than two years. 

Subsistence payments may be made gross on the Visiting Research Fellow form if the following conditions are met:
There is no contract of employment.
There is no contract for services, whether written or verbal.
The visiting Research Fellow does not undertake any duties (apart from research) normally performed by staff of GNEI.
The grant is paid to cover living and travel expenses.
The visiting Research Fellow is here for a maximum of two years.
The visiting Research Fellow form must clearly state that the payment is for travel and subsistence of an overseas visiting researcher.
The individual’s home country and institution address is included on the form.
The source of funding is stated on the form.
If the visiting Research Fellow does undertake any teaching duties or hold seminars, then a proportion of the payments made to them must be attributed to these duties. Payments may still be made gross but they must be reported to the HMRC. Details of all such payments in any tax year (6 April to 5 April) must be returned to the Taxation and Commercial Accountant in Finance by 30 April each year.


Annex 5 - Childcare and Caring Responsibility costs
Various funders now allow childcare or caring responsibility costs to be claimed as allowable expenses if the costs are incurred for working outside normal working patterns (e.g. childcare or eldercare costs while the primary carer attends a conference outside usual contracted hours).

Please refer to your funder’s terms and conditions or obtain permission from them prior to incurring the expenditure. If you require further guidance please contact Award Services, Research and Innovation Services award-services@gnei.ac.uk

Criteria
The following criteria will need to be met for claims to be approved through GNEIExpenses:
Staff must be on a grant under the terms of which such a claim is expressly allowable and eligible for reimbursement.
There must be an auditable record such as conference attendance confirmation and receipts for the cost of the childcare / caring responsibility.




"""

print(len(context_full))

context_travel = """

GNEI Expenses Policy
1. Introduction 
This Expenses Policy (‘the Policy’) provides guidance to all individuals claiming reimbursement of reasonable expenses incurred in connection with GNEI business.

2. General Principles
GNEI employees and all others engaged in GNEI activity (collectively claimants) will be reimbursed for the actual cost of expenses incurred, wholly, exclusively and necessarily in the performance of GNEI activity as prescribed in this policy.
The expense must be justifiable and reasonable according to the information in this policy, and the claim should always be prepared honestly, legally and responsibly. Any breach of the policy could lead to disciplinary action, up to and including dismissal.
Whilst this policy aims to provide comprehensive guidance on reimbursable out-of- pocket expenditure, it is recommended that staff still seek advance approval from their budget holder or department manager in any situation where interpretation of this policy is in doubt.
Employees should be able to procure the majority of goods and services required for GNEI business using the standard purchase to payment procedures using the GNEIFinance system.
It is the responsibility of authorised signatories to ensure that this expenses policy is upheld; and it should be noted that individual claims falling outside of the policy may be subject to review and/or rejection by Finance.
Electronic images of original receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used should include details of what goods or services have been purchased. Original paper receipts do not need to be stored locally, unless they relate to sponsored research and it is explicit in the terms of the grant that original paper receipts need to be retained.
When making a claim for a group, the most senior person present should pay for the expenditure and make the claim.
Expenses should be submitted as soon as possible after they have been incurred. Claims must be made within three months. Approval should be gained from the relevant School or PS Head of Finance if the claim is over three months old.
It costs a fixed amount to process any claim, so claimants should endeavour not to submit claims for less than £33, unless they represent the total of expenses in a three-month period.
If a claimant inadvertently makes an error with an expense claim and has been reimbursed, GNEI will recover the amount from them.
Expenses must not be used as a way of rewarding people or encouraging them to work in remote locations.
GNEI will not meet the cost of expenses for spouses, partners, other family members or friends of GNEI staff.
Claims are checked by Accounts Payable staff and any deemed to be fraudulent will be investigated and referred to GNEI’s Internal Auditors. If a claimant is found to have submitted a fraudulent claim they will face disciplinary proceedings, leading to penalties up to and including summary dismissal.



3. Travel
This section details what can be claimed as travel expenses and provides information on specific modes of transport and other expenses associated with travel.

General guidance for travelling on GNEI business
The cost of business travel, that is, journeys away from your normal place of work while undertaking GNEI business can be claimed.
In line with the GNEI travel policy, all business trips in the UK or abroad must be authorised in writing in advance by the GNEI budget holder responsible for making the funds available. It is recommended that this is attached to the claim.
The budget holder / manager must agree that the travel is necessary and that alternative methods such as conference call, video conference, phone or email cannot be used or are not appropriate. Consideration should also be given as to how to minimise the environmental impact of travel.
This authorisation ensures that anticipated costs are in line with departmental plans, are eligible under the terms of any associated funding, and that subsequent expense claims can be submitted with confidence, subject to the provisions of this policy.
GNEI has preferred travel suppliers for airline, hotel and car hire bookings details on these suppliers can be found on the Procurement website. These suppliers should be used for travel bookings where possible.

Private travel costs
The following cannot be claimed:
Travel between home and normal place of work.
Business travel broadly similar to the claimant’s normal commute.
Recreational travel and accommodation at or near the business travel destination.
Travel or accommodation for family or friends accompanying the claimant on the business journey.

Rate of exchange
If a credit card was used to pay for overseas expenses, the rate charged by the credit card issuer can be claimed. A copy of the statement should be used to evidence the rate used.
For employees GNEIExpenses will use a default rate for translating foreign currency transactions; this can be overridden if different to the rate at the time of expenditure and fluctuations will be allowed within a set tolerance. If the automatic rate is overridden, a copy of the bank statement should be included with the claim to back up the rate used.
Buying foreign currency is not a genuine business expense and cannot be claimed.

Travel Insurance
Travel Insurance needs to be activated before traveling overseas, travelling in the UK for fieldtrips or travelling on a business trip where there is an overnight stay or air travel involved.
Insurance can be activated by completing a travel insurance request form, which can be found in the Insurance Section on the Finance website.
The Insurance team will issue a cover note and provide details of the policy. There is no cost to departments for travel insurance. For regular travellers the Insurance Section can issue annual cover notes on request.
If a claimant has an existing medical condition and is not travelling against their doctors’ orders, this is covered by the GNEI Travel Insurance Policy at no extra cost.
Personal travel insurance cannot be claimed as GNEI already provides cover for business travel, please refer to Annex 2 for further information on this exclusion.

Public Transport
Journeys made on rail, bus, ferry, river boat, underground, metro or tram services – can be claimed.
The most economical method of travel should be used. For example, if the claimant uses public transport in London regularly, they should use an Oyster card rather than paying for single journeys by cash.

Train journeys
These should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel.
If the claimant does travel first class, an explanation must be included on the claim and if necessary the claim will be passed to the relevant School or PS Head of Finance for approval.
Trains should be booked in advance where possible via GNEI's approved travel provider or similar rail travel provider as this is the most cost effective option. Tickets can then be collected at the relevant station or sent via post.

Tube and bus journeys in London
If a London Transport Oyster or contactless credit/debit card is used for business journeys, the details of each individual journey should be entered on the claim, being specific about locations, date and times and purpose of journey.
Only the relevant journey should be claimed, not the whole top-up cost, unless the whole top-up will be used for GNEI business travel. Receipts are not necessary for individual journeys but are required if claiming for the whole top- up fee.

Receipts
Relevant tickets or documentation should be kept and attached to the expense claim electronically.
Claimants who have a rail or Oyster season ticket enabled to use the underground can only claim for travel if extra expenses are incurred that are not covered by the season ticket and can provide a receipt as proof of the expense.
Ticket machines will provide receipts if the ticket is retained by the automatic barrier at the end of a journey.

Taxis
Taxi fares can only be claimed for journeys where:
It is clearly the most efficient and cost-effective method of transport.
Alternative methods of transport are impractical due to pregnancy, disability, illness or injury, luggage or similar.
A member of staff is working in the office very late, i.e. after 11pm.
Prior approval has been given, where feasible, from the budget holder or departmental administrator/manager.

Business Mileage
If a car, motorcycle or bicycle has been used on a business trip the costs can be claimed using a mileage allowance – that is, a sum of money for every business mile travelled. GNEI does not reimburse claims based on actual petrol receipts.
GNEI has standard rates for these allowances, depending on the circumstances surrounding the trip (see below).
Only additional costs incurred, above what would normally be spent for commuting to work (which are not eligible as a business expense) can be claimed.
A car should only be used if it is cheaper and more efficient than public transport. Cars should be shared with other staff members wherever possible.
The claim should state the start and end points of the journey, along with the number of miles being claimed.
Mileage is paid at the HMRC approved mileage allowance rates:
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter.
Motorcycles - 26p per mile.
Bicycles - 22p per mile.

Car/bicycle/motorcycle insurance
It is important that adequate personal insurance cover for business use is in place, as cars, bicycles or motorcycles belonging to claimants are not covered by GNEI’s insurance. Personal insurance cannot be claimed, see Annex 2 for further information.
If a staff member uses or allows another member of staff to use a vehicle on GNEI’s business that is insured for third party risks only, GNEI will not be held liable for any of the following:
damage or repairs to the vehicle
property lost or damaged
personal injuries sustained or caused during or as a result of such use.

Car sharing
Only the driver can claim mileage for a journey where two or more GNEI staff share a car. The following cannot be claimed:
Insurance cover for private cars.
Accident or breakdown recovery costs, such as AA membership.

Air Travel
Air travel should be booked using GNEI’s preferred travel provider where possible.
GNEI prefers lower fares, based on flight arrival and departure times which cannot be changed. Where there is a need for flexibility in departure or arrival times, a different ticket option will have to be requested. Flexible tickets for flights under three hours require approval by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.
Membership of a frequent traveller scheme and potential air miles benefits must not deflect from using the most cost-effective airline option for travel.
GNEI does not meet the travelling costs of:
Employees of contractors
Spouses, partners, other family members or friends.
Classes of travel:
For flights of five hours or less, staff should book restricted tickets in economy.
For flights lasting five hours or longer staff may travel premium economy.
Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class. This must be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Toll charges
Toll charges paid for in the course of a business journey can be claimed. Toll bridges or roads rarely give receipts; therefore the name of the toll bridge or road should be included on the expense claim.

Congestion charges
GNEI will not reimburse the cost of any congestion charge for staff or visitors except in exceptional circumstances, e.g. where a member of staff has to collect/deliver equipment to a location and the journey could not be completed on public transport. This must be approved in advance by the relevant School/PS Head of Finance.

Car Parking
Reasonable parking costs on business visits and journeys away from the office can be claimed.
Hotel parking costs can be claimed if they are charged separately on the bill.
If a claimant has to drive to their place of work as part of a business journey, e.g. to collect equipment, parking costs can be claimed. Details of the reason should be included with the claim.
The cost of parking at an entertainment venue cannot be claimed.
VAT can be reclaimed (where it's charged) on parking fees which cost less than £28 even if the claimant is not able to get a VAT receipt.

Car Hire
Approval must be obtained from the budget holder or departmental administrator/manager before a car is hired.
If hiring a car is the cheapest mode of travel, it should be booked through GNEI's approved car hire provider (for UK or overseas).
The car should only be used for business purposes.
Rates are inclusive of CDW (Collision Damage Waiver), TPI (Third Party Insurance) and PAI (Personal Accident Insurance).
Any petrol bought solely for business purposes can be claimed. A VAT receipt should be obtained for purchases, a credit card receipt is not sufficient.

Group Travel
This is in relation to a staff conference or student field trip. Prior approval from the budget holder or departmental administrator/manager should be obtained before booking a group trip.
The same rules apply for each category of travel and overnight costs as for travelling as an individual.
All travel (including coach hire) and overnight costs should be booked in advance.
GNEI's approved travel provider provides a group travel & conference booking facility, where all elements of the trip can be arranged. This can be used for arranging a business conference, as well as student field trips.
Please ensure adequate insurance is in place for all staff and students on the trip. Travel insurance needs to be activated prior to the trip by using the Insurance section of the Finance website.

Travel Incidentals
Claimants can claim the following goods or services if they are related to the business purpose of the trip:
The cost of obtaining a visa for working abroad on GNEI business. These can be obtained from GNEI's approved travel provider.
The cost of any vaccinations needed for working abroad.
There may be occasions when a claimant needs a second passport, for example when travel to one country could lead to immigration difficulties in travelling on business to some other countries. GNEI will pay this expense.
Approval must be obtained in advance from the budget holder or departmental administrator/manager.


"""

context_accom = """

3. General Principles
GNEI employees and all others engaged in GNEI activity (collectively claimants) will be reimbursed for the actual cost of expenses incurred, wholly, exclusively and necessarily in the performance of GNEI activity as prescribed in this policy.
The expense must be justifiable and reasonable according to the information in this policy, and the claim should always be prepared honestly, legally and responsibly. Any breach of the policy could lead to disciplinary action, up to and including dismissal.
Whilst this policy aims to provide comprehensive guidance on reimbursable out-of- pocket expenditure, it is recommended that staff still seek advance approval from their budget holder or department manager in any situation where interpretation of this policy is in doubt.
Employees should be able to procure the majority of goods and services required for GNEI business using the standard purchase to payment procedures using the GNEIFinance system.
It is the responsibility of authorised signatories to ensure that this expenses policy is upheld; and it should be noted that individual claims falling outside of the policy may be subject to review and/or rejection by Finance.
Electronic images of original receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used should include details of what goods or services have been purchased. Original paper receipts do not need to be stored locally, unless they relate to sponsored research and it is explicit in the terms of the grant that original paper receipts need to be retained.
When making a claim for a group, the most senior person present should pay for the expenditure and make the claim.
Expenses should be submitted as soon as possible after they have been incurred. Claims must be made within three months. Approval should be gained from the relevant School or PS Head of Finance if the claim is over three months old.
It costs a fixed amount to process any claim, so claimants should endeavour not to submit claims for less than £33, unless they represent the total of expenses in a three-month period.
If a claimant inadvertently makes an error with an expense claim and has been reimbursed, GNEI will recover the amount from them.
Expenses must not be used as a way of rewarding people or encouraging them to work in remote locations.
GNEI will not meet the cost of expenses for spouses, partners, other family members or friends of GNEI staff.
Claims are checked by Accounts Payable staff and any deemed to be fraudulent will be investigated and referred to GNEI’s Internal Auditors. If a claimant is found to have submitted a fraudulent claim they will face disciplinary proceedings, leading to penalties up to and including summary dismissal.

5. Overnight costs and allowances
Such costs are subject to the requirement for pre-approval in section 4.3 above.

Hotels
In line with the GNEI travel policy, Hotels must be booked via GNEI's approved travel provider for both overseas and UK travel, with recovery through expenses only permitted in those cases set out in Section 3.1 of that policy and subject to an appropriate level of advance approval as set out in section 3.2. The approval obtained must be attached to submitted expense claims alongside receipts for the room cost, breakfast, dinner and other eligible incidentals.
Hotel city tax: from time to time, additional local city tax charges may apply for accommodation. These can be reclaimed via expenses.

UK Hotels
As stipulated by HMRC, the cost of accommodation at or near the normal place of work does not qualify as business travel; any claim would be subject to tax on the individual as a reimbursement of personal expenditure. Such costs may not be claimed, and staff are asked to bear this in mind when making their travel arrangements.
The cost of a hotel in the UK should not exceed £210 including VAT (room only) in London or £140 including VAT (room only) outside of London. If a hotel cost is higher than these limits, this should be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Overseas hotels
The cost of overseas accommodation may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this in mind, the University does not look to prescribe fixed limits for overseas accommodation but instead provide guidance on what a reasonable claim would be so that claimants and approvers are able to make a fully informed assessment.
In the first instance, employees should expect to be reimbursed for hotels that are equivalent to the most appropriate GBP rate for UK travel detailed in section 5.5. For overseas stays the expectation of the University is that an employee should book accommodation that is equivalent to a UK 3-star level. For areas where there is deprivation or a high security risk then the rating may be increased to 4-star.
Employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit nor an automatic entitlement but should be considered a point of reference only.
Employees should be aware however that claims materially in excess of the HMRC rates are likely to be subject to challenge either during the approval process or before settlement by the central processing teams. Where this is expected to be the case, claimants should ensure that clear justification and confirmation of pre-approval from the relevant budget holder is attached to the claim as per Section 4.3.
Where attendance at an international conference requires specific accommodation (for example by virtue of a fixed attendance package or as a necessary means of realising the benefits of the conference), this factor should also be documented with pre-approval attached to the claim.
Incidentals
All necessary incidentals (including brief personal calls home) should be receipted and claimed:
Newspapers, bar drinks, mini bar, hotel video and health and fitness facilities cannot be claimed.
Laundry costs can only be claimed if absolutely necessary, the cost is reasonable and appropriate and the stay away is for at least 5 consecutive nights.

Telephone, Fax and Internet Charges
The cost of business calls, fax and internet access charges for business use, as long as they are included on an itemised bill, can be claimed.
When working overseas a telephone card can be purchased as this reduces the cost of local calls, the cost of this instead of calls from hotels can be claimed.

Meals
When travelling on GNEI business, employees are entitled to recover the cost of reasonable subsistence, namely breakfast, lunch, and dinner. Meals cannot be claimed if already included in hotel accommodation rates being claimed for.
If several members of staff have a meal together, the most senior person should pay and submit the expense claim. In such a case no-one other than the most senior person should claim an allowance for the meals. Please ensure that it is made clear in the justification field of GNEIExpenses that the receipt is covering more than one employee’s meal, and each employee name (covered by the receipt) should be listed.
The following rates include the cost of alcoholic beverages.

UK Meals
Breakfast. If breakfast is not included in the hotel room rate, the cost can be claimed, which should be receipted and claimed as part of the overnight stay to a maximum of £12 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for breakfast.
Lunch. If the claimant is working away from their normal place of work, lunch can be claimed to a maximum of £18 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for lunch.
Evening meal. The cost of an evening meal can be claimed which should be receipted and claimed as part of the overnight stay to a maximum of £36 including VAT and service. If it is not possible to obtain a receipt, then £17 may be claimed for dinner.
24-hour rate: Where UK travel includes an overnight stay and incorporates breakfast, lunch and dinner a single claim of up to £66 receipted (£28 un-receipted) may be made. In these cases, individual meal rates need not be applied, and the claim will be assessed on an aggregate basis in order to allow for greater flexibility. This 24-hour rate will not apply where any one of the meals is included within a room rate, and claimants must still provide receipts for each separate meal. Only actual spend will be reimbursed and all claims remain subject to the overriding principles of reasonableness and value for money.

Overseas Meals
The cost of meals may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this mind, the University does not look to prescribe fixed limits for overseas meals but instead provide guidance on what a reasonable claim would be.
In the first instance, employees should expect to be reimbursed for meals that are equivalent to the standard of meal that can be purchased in the UK within the set GBP rates for UK travel detailed in sections 5.18 to 21.
As an additional location-specific guide, employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit for claims (noting that they’re updated infrequently) but nor do they represent an automatic entitlement. The rates should be considered a point of reference only and all claims must uphold the general principles set out above.
In line with section 3 above, employees will only be reimbursed for actual costs incurred as evidenced by original receipts. Where receipts for overseas subsistence cannot be obtained, employees may claim at the unreceipted GBP rates set out in sections 5.18 to 5.21 up to a maximum of £28 per day for three meals.

Rented Accommodation (This section applies to GNEI staff and students only, the policy for visiting Research Fellows is detailed in Annex 4).
Long-term assignments
If a member of staff is working away from home on a longer-term assignment – usually for six months or more – the budget holder may require them to rent a property rather than stay in a hotel.
The lease should be taken out in the employee’s name as they will be the sole occupant of the property; prior approval from budget holder or departmental administrator/manager must be obtained before the lease is agreed.
The monthly rent, council tax, service charges, TV licence and utility bills (including telephone line rental) including VAT can be claimed. Alternatively, GNEI can pay the rent directly on invoice via Accounts Payable.
If several colleagues are working away from home working on the same project and sharing a property, only one of the staff should claim expenses for the cost of the accommodation and associated bills.

Relocation Expenses
Please refer to the Relocation Expenses Guide for further details.

"""

print(len(context_accom))

44769
11553


In [7]:
def data_format(example, context):
    # Format system
    system1 = f"""A <<<POLICY>>> and a <<<SCENARIO>>> will be provided, you are GREAT at analysing the <<<SCENARIO>>> according to the <<<POLICY>>>. Your answer MUST and and CAN ONLY be a valid json format, having 4 text fields: Classification T1, Reimbursement Amount, Classification T2, and Reasons. Your answer should follow these structures: \n 1) "Classification T1" :you MUST choose 'Policy Violated' or 'Policy Not Violated' for the activity in the <<<scenario>>> \n \n 2) "Reimbursement Amount" : Answer One number directly that according to the Policy the total amount of money can be reimbursed \n 3) "Classification T2" : Compare to the requested reimbursement amount in the scenario, and answer if the claim is 'Fully Reimbursable'/'Partially Reimbursable'/'Not Reimbursable'/'Further Clarification Required'. \n 4) give reasons for your choices according to the <<<POLICY>>>.\n\n
    """

    system2 = f"""<<<<<Policy Start:{context}
    \n\n\nPolicy End>>>>>\n\n
    """

    # Format instruction
    prompt = f"""<<<<<Scenario Start:\n\n{example['Prompt']}\n\nScenario End>>>>>"""

    # Format chosen answer
    chosen = f"""Answer: {{"Classification T1": {example['Classification T1']},
    "Reimbursement Amount": £{example['Amount']},
    "Classsifcation T2": {example['Classification T2']},
    "Reasons": ...}} 
    <eos>\n"""

    # Format rejected answer
    rejected = f"""Answer: {{"Classification T1": {example['rejected_T1']},
    "Reimbursement Amount": £{example['rejected_Amount']},
    "Classsifcation T2": {example['rejected_T2']},}} 
    <eos>\n"""

    return {
        "prompt": system1 + system2 + prompt,   # sequence should follow inference patterns and context generally come before related questions
        # "prompt": system1 + prompt,           # option for non-policy ft
        "chosen": chosen,
        # "rejected":"",
        "rejected":rejected,
    }

In [8]:
# Save columns
original_columns = tr_data.column_names

# Format dataset
dataset = tr_data.map(
    lambda x: data_format(x,context = context_accom),
    remove_columns=original_columns # remove un-used columns for training process
)

print(data_frame['Prompt'][5])
print(dataset[5]['chosen'])
print(dataset[5]['rejected'])
dataset = dataset.train_test_split(test_size=0.05)
dataset

Map: 100%|██████████| 200/200 [00:00<00:00, 5068.43 examples/s]

Representing GNEI, I journeyed from London to Madrid for a conference, incurring a flight expense of £92. Furthermore, the bank levied a £15 charge for currency exchange during the transaction. Could you please confirm the reimbursement amount?
Answer: {"Classification T1": Policy Not Violated,
    "Reimbursement Amount": £750.0,
    "Classsifcation T2": Fully Reimbursable,
    "Reasons": ...} 
    <eos>

Answer: {"Classification T1": Policy Violated,
    "Reimbursement Amount": £432.68,
    "Classsifcation T2": Partially Reimbursable,} 
    <eos>



DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 190
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 10
    })
})

In [8]:
test_frame = pd.read_excel(file_path,sheet_name=2, engine='openpyxl')

data_test = test_frame[test_frame["Category"] == "Accomodation and Sustenance"]
data_test.reset_index(drop=True, inplace=True)

In [9]:
def get_responses(prompt_num, data_frame, context):
    """
        prompt_num: int: how many data points in data_frame involved
        data_frame: pdDataframe: provides a "Prompt" column containing scenarios
        context:    str: the policy script to be included in the prompt

        return:
        pdDataframe: model_original_resp: a df containing trimmed responses in a column manner
        """
    model_original_resp = []
    for i in range(prompt_num):
        # iterating through all data points
 
        prompt = f"""A <<<POLICY>>> and a <<<SCENARIO>>> will be provided, you are GREAT at analysing the <<<SCENARIO>>> according to the <<<POLICY>>>. Your answer MUST and and CAN ONLY be a valid json format, having 4 text fields: Classification T1, Reimbursement Amount, Classification T2, and Reasons. Your answer should follow these structures: \n 1) "Classification T1" :you MUST choose 'Policy Violated' or 'Policy Not Violated' for the activity in the <<<scenario>>> \n \n 2) "Reimbursement Amount" : Answer One number directly that according to the Policy the total amount of money can be reimbursed \n 3) "Classification T2" : Compare to the requested reimbursement amount in the scenario, and answer if the claim is 'Fully Reimbursable'/'Partially Reimbursable'/'Not Reimbursable'/'Further Clarification Required'. \n 4) give reasons for your choices according to the <<<POLICY>>>.\n\n

        <<<<<start examples:

        <<<<<example one:
        scenario: I used a TFL travel card to travel from home to the GNEI work station every day for a week and it cost me £50. How much can I reimburse?

        Answer:{{"Classification T1" : "Policy Violated",
                "Reimbursement Amount" : £0,
                "Classification T2" : "Not Reimbursable",
                "Reasons":"According to the GNEI Expenses Policy, travel between home and normal place of work (i.e., the GNEI campus) cannot be claimed. Thus, this expense cannot be reimbursed."}};

        <<<<<example two:
        scenario: I booked a taxi from the GNEI work station to my home yesterday night. The total cost was £25. I left the campus at 11:30pm. Can I expense this charge?

        Answer:{{"Classification T1" : "Policy Not Violated",
                "Reimbursement Amount" : £25,
                "Classification T2" : "Fully Reimbursable",
                "Reasons":"The GNEI Expenses Policy states that taxi fares can be claimed for journeys where a member of staff is working in the office very late, specifically after 11pm​​. Therefore, your taxi fare under these circumstances falls within the allowable expenses according to the policy, and total reimbursable amount is £25."}};
        
        <<<<<example three:
        scenario: I booked a taxi from the GNEI campus to my home yesterday night. The total cost was £35. I left the campus at 9:30pm. Can I expense this charge?

        Answer:{{"Classification T1" : "Policy Violated",
                "Reimbursement Amount" : £0,
                "Classification T2" : "Further Clarification Required",
                "Reasons":"Based on the GNEI Expenses Policy, taxi fares can only be claimed for specific reasons, such as if a member of staff is working in the office very late, specifically mentioned as after 11pm. Since you left the campus at 9:30pm, your taxi fare does not meet the criteria set out under the "Taxis" section for allowable taxi expenses.
                Therefore, based on the information provided in the GNEI Expenses Policy, you cannot expense the £35 taxi charge for a trip from the GNEI campus to your home that occurred at 9:30pm."}};
        
        <<<<<example four:
        scenario: I drove to Cambridge for a business trip along with another GNEI colleagues. The total distance covered was 60 miles. I accidentally hit the sidewalk and damaged the side of the car. The total cost of the repairs was £100. How much can I claim in expenses?  

        Answer:{{"Classification T1" : "Policy Violated",
                "Reimbursement Amount" : £30,
                "Classification T2" : "Partially Reimbursable",
                "Reasons":"For your business trip to Cambridge, driving a car for a total distance of 60 miles with another GNEI colleagues, you can claim expenses based on the GNEI Expenses Policy as follows:
                - **Mileage Claim**: According to section **4.32**, mileage for cars is reimbursed at a rate of 50p per mile for the first 11,000 miles in a tax year. Therefore, for 65 miles, the claim would be 60 miles x £0.50/mile = £30.
                - **Damage to the Vehicle**: Section **4.34** specifies that GNEI will not be held liable for damage or repairs to the vehicle used on GNEI’s business. Thus, the cost of repairs due to the accident, totaling £100, cannot be claimed under the policy.
                **Documentation and Approvals Required:**
                1. **Mileage Documentation**: Include details such as the start and end points of the journey, the purpose of the business trip, and the total number of miles claimed.
                2. **Approval for Business Travel**: Ensure that the business trip was pre-approved as per GNEI's travel policy requirements.
                3. **Vehicle Damage**: Since the policy excludes claims for vehicle damage or repairs, there is no requirement for documentation or approval in this context.

                **Total Amount That Can Be Expensed: £30** (for mileage only, as vehicle repair costs are not covered)."}};

        examples End>>>>>

        <<<<<Policy Start:{context}
        \n\n\nPolicy End>>>>>\n\n<<<<<Scenario Start:\n\n{data_frame['Prompt'][i]}\n\nScenario End>>>>>"""
        
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        # input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        # print("tokenization completed",input_ids['input_ids'].shape)

        #get "Answer" token ID in the tokenizer, forcing the model to start answering with "Answer" and thus give a json output
        start_token_id = tokenizer.convert_tokens_to_ids("Answer")
       
        outputs = model.generate(**input_ids,max_new_tokens = 400, min_new_tokens = 5,
                                 #do_sample = False, temperature = 5, top_k = 0.9, 
                                 repetition_penalty = 1.2,
                                 forced_bos_token_id = start_token_id) # alter max number of tokens here
        
        # trim the output by removing prompt
        model_response = tokenizer.decode(outputs[0])

        trimmed_output = model_response[len(prompt)+5:]

        # collecting responses
        model_original_resp.append(trimmed_output)

        if ((i+1)%10 == 0):
                print(f"the",(i+1),"th prompt responded")
        #print(f"the",(i+1),"th prompt responded")
    return model_original_resp


In [11]:
prompt_num = len(data_test)
composed_resp = []
for i in range(1):
    model_original_resp = get_responses(prompt_num, data_test, context_accom)    
    composed_resp.extend(model_original_resp)

df_original_resp = pd.DataFrame(composed_resp, columns=['original response'])
# df_original_resp.head(5)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyboardInterrupt: 

In [50]:
prompt_num = len(data_test)
composed_resp = []
for i in range(1):
    model_original_resp = get_responses(prompt_num, data_test, context_full)    
    composed_resp.extend(model_original_resp)

df_original_resp2 = pd.DataFrame(composed_resp, columns=['original response'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


the 10 th prompt responded


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


the 20 th prompt responded


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


the 30 th prompt responded


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


the 40 th prompt responded


In [11]:
import re

def extract_labels(text):
    # label patterns. Further wirk to extract label from certain key pairs only
    pattern1 = r': "(Policy\sViolated|Policy\sNot\sViolated)"'
    pattern2 = r': £\s*([\d,]+\.?\d*)'  # extract the number after £ 
    pattern3 = r': "(Fully\s+Reimbursable|Partially\s+Reimbursable|Not\sReimbursable|Further\sClarification\sRequired)"'
    
    # create match
    match1 = re.search(pattern1, text)
    match2 = re.search(pattern2, text)
    match3 = re.search(pattern3, text)
    
    amount = 0
    # processing Amount label, deleting comma
    if match2:
        try:
            # Remove commas and convert to float
            temp = match2.group(1)
            amount = float(temp.replace(',', ''))
        except ValueError:
            amount = 0 

    # find match
    label1 = match1.group(1) if match1 else None
    #label2 = amount.group(1) if match2 else "0"
    label3 = match3.group(1) if match3 else None
    # give a format marker , 1 if there's at least 1 classification label missing and 0 if not 
    label_format = 1 if label1 is None or label3 is None else 0 

    return label1, amount, label3, label_format

def extract_all_labels(scenario, response_list, num):
    label_list = []
    for i in range(num):
        text = response_list['original response'][i]
        label1, label2, label3,label_format = extract_labels(text)
        label_list.append([scenario[i], text, label1, label2, label3,label_format])
    return label_list

In [51]:

label_list = extract_all_labels(data_test['Prompt'], df_original_resp, prompt_num)

df_label = pd.DataFrame(label_list, columns=['Scenario', 'Text', 'Classification T1', 'Reimbursable Amount', 'Classification T2','Format Compliance'])


In [12]:
# Evaluate
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
def evaluate(response_df, prompt_df):
    T1_accurate = 0
    T2_accurate = 0
    amount_accurate = 0
    for i in range(len(response_df)):
        pre_label = response_df[['Classification T1', 'Reimbursable Amount', 'Classification T2','Format Compliance']].iloc[i]
        ori_label = prompt_df[['Classification T1', 'Amount', 'Classification T2']].iloc[i]

        if pre_label['Classification T1'] == ori_label['Classification T1']:
            T1_accurate += 1
        if float(pre_label['Reimbursable Amount']) == float(ori_label['Amount']):
            amount_accurate += 1
        #print(f"Predicted: {pre_label['Reimbursable Amount']}, should be {ori_label['Amount']}")
        if pre_label[ 'Classification T2'] == ori_label['Classification T2']:
           T2_accurate += 1
    T1_accuracy = T1_accurate/len(response_df) 
    T2_accuracy = T2_accurate/len(response_df)
    amount_accuracy = amount_accurate/len(response_df)
    format_accuracy = (1 - response_df['Format Compliance'].sum()/len(response_df)) * 100

    # F1 score calculation
    # To calculate F1 requires labelled inputs
    encoder = LabelEncoder()
    a = encoder.fit_transform(['Policy Violated', 'Policy Not Violated', 'None'])
    f1_T1 = f1_score(encoder.transform(prompt_df['Classification T1'][:len(response_df)]), encoder.transform(response_df['Classification T1']), average='weighted')

    print(a,encoder.transform(response_df['Classification T1']).sum())

    encoder.fit(['Fully Reimbursable','Partially Reimbursable','Not Reimbursable','Further Clarification Required','None'])
    #f1_amount = f1_score(prompt_df['Reimbursable Amount'], response_df['Reimbursement Amount'], average='weighted')
    f1_T2 = f1_score(encoder.transform(prompt_df['Classification T2'][:len(response_df)]), encoder.transform(response_df['Classification T2']), average='weighted')

    print(f"T1 acc: {T1_accuracy}, T1 F1 Score: {f1_T1};\nT2 acc: {T2_accuracy}, T2 weighted F1 Score: {f1_T2};\namount acc: {amount_accuracy}, \nformat acc: {format_accuracy} ")



In [52]:

evaluate(df_label, data_test)

[2 1 0] 45
T1 acc: 0.6, T1 F1 Score: 0.5826923076923076;
T2 acc: 0.45, T2 weighted F1 Score: 0.4132006882006882;
amount acc: 0.3, 
format acc: 95.0 


In [53]:

label_list2 = extract_all_labels(data_test['Prompt'], df_original_resp2, prompt_num)

df_label2 = pd.DataFrame(label_list2, columns=['Scenario', 'Text', 'Classification T1', 'Reimbursable Amount', 'Classification T2','Format Compliance'])
print("now full policy:")
evaluate(df_label2, data_test)

now full policy:
[2 1 0] 51
T1 acc: 0.6, T1 F1 Score: 0.5786666666666667;
T2 acc: 0.425, T2 weighted F1 Score: 0.37120708748615727;
amount acc: 0.3, 
format acc: 100.0 


In [10]:
# New Code
from transformers import AutoTokenizer, AutoModelForCausalLM
base_model = model_name

torch_dtype = torch.float16

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        # load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

# Reduce VRAM use
model.config.pretraining_tp = 1
# model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token


# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.11s/it]


In [58]:
p = dataset['test']['prompt'][0]
input_po = tokenizer(p, return_tensors="pt").to("cuda")
# input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
print(input_po['input_ids'].shape)

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0,
)

torch.Size([1, 2903])


In [59]:
orpo_args = ORPOConfig(
    learning_rate=2e-5,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length= 3300,
    max_prompt_length= 3072,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=20,
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=1,
    bf16=True,
    warmup_steps=20,
    report_to="wandb",
    output_dir="./results/",

    save_steps=20,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Ensures the best model is loaded at the end
)

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,

    callbacks=[early_stopping],
)

/home/uceehuf/.local/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 10/10 [00:00<00:00, 40.43 examples/s]
/home/uceehuf/.local/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [60]:
new_model = 'ORPO-Mistral-7b'
orpo_trainer.train()
orpo_trainer.save_model(new_model)

/home/uceehuf/.local/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
20,1.579700,1.538651,9.210200,1.086000,0.543000,-0.125215,-0.241181,1.000000,0.115966,-2.411807,-1.252149,-3.154190,-3.153484,1.516210,-0.224416,1.403116
40,0.436300,0.384155,9.199100,1.087000,0.544000,-0.038132,-0.357635,1.000000,0.319503,-3.576350,-0.381322,-2.796489,-2.795533,0.382410,-0.017449,4.488202
60,0.035900,0.039194,9.214500,1.085000,0.543000,-0.011466,-0.425172,1.000000,0.413705,-4.251717,-0.114663,-2.175448,-2.173084,0.038939,-0.002545,7.178544
80,0.032100,0.034273,9.202300,1.087000,0.543000,-0.008884,-0.430047,1.000000,0.421163,-4.300467,-0.088840,-2.507337,-2.505198,0.034095,-0.001780,7.396731
100,0.030900,0.032325,9.210700,1.086000,0.543000,-0.008177,-0.443117,1.000000,0.434940,-4.431169,-0.081767,-2.581347,-2.579115,0.032182,-0.001434,7.780704
120,0.029800,0.030870,9.205700,1.086000,0.543000,-0.008079,-0.455634,1.000000,0.447555,-4.556339,-0.080792,-2.587422,-2.585092,0.030743,-0.001268,8.068693
140,0.026400,0.029347,9.206600,1.086000,0.543000,-0.008014,-0.462234,1.000000,0.454220,-4.622344,-0.080142,-2.581806,-2.579388,0.029227,-0.001205,8.299503
160,0.028300,0.027884,9.210900,1.086000,0.543000,-0.007796,-0.466496,1.000000,0.458700,-4.664957,-0.077960,-2.557693,-2.555197,0.027773,-0.001114,8.348478
180,0.024300,0.026634,9.200200,1.087000,0.543000,-0.007820,-0.469638,1.000000,0.461817,-4.696378,-0.078205,-2.551430,-2.548878,0.026525,-0.001094,8.434547
200,0.024600,0.026039,9.200800,1.087000,0.543000,-0.007780,-0.470929,1.000000,0.463149,-4.709290,-0.077804,-2.542892,-2.540333,0.025932,-0.001067,8.454558


Checkpoint destination directory ./results/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/uceehuf/.local/lib/python3.11/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66957041-2b449c895f5429580e1b24a3;da830b73-3931-48aa-b3ec-6263cf1c8e19)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/home/uceehuf/.local/lib/python3.11/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that the vocabulary was not modified.
  warnings.warn(
/home/uceehuf/.local/lib/python3.11

Old code NANs happening

In [11]:

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False, #True,
)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        # load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

# Reduce VRAM use
model.config.pretraining_tp = 1
# model.gradient_checkpointing_enable()

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


In [12]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

In [13]:
orpo_args = ORPOConfig(
    learning_rate=2e-5,
    beta=0.1,
    lr_scheduler_type="linear",
    #max_length= ,
    max_prompt_length=5120,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.05,
    logging_steps=1,
    warmup_steps=20,
    report_to="wandb",
    output_dir="./results/",
)

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)

/home/uceehuf/.local/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/uceehuf/.local/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 10/10 [00:00<00:00, 46.98 examples/s]
/home/uceehuf/.local/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration

In [14]:
orpo_trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junyu-zhu-23 (ucl_student). Use `wandb login --relogin` to force relogin


/home/uceehuf/.local/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
5,0.000000,nan,49.818700,0.201000,0.201000,nan,nan,0.000000,nan,nan,nan,-3.133734,-3.133734,2.068200,nan,nan
10,0.000000,nan,49.846200,0.201000,0.201000,nan,nan,0.000000,nan,nan,nan,-3.133734,-3.133734,2.068200,nan,nan


KeyboardInterrupt: 

In [24]:
import gc

# Flush memory
del orpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

In [61]:
new_model = 'ORPO-Mistral-7b'
orpo_trainer.save_model(new_model)

base_model = model_name
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

model, tokenizer = setup_chat_format(model, tokenizer)
# Merge adapter with base model
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

/home/uceehuf/.local/lib/python3.11/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-669599c7-581eec9118dbcbc51117cb84;9a1f2ad8-343b-4cad-b1c3-2b02e027c535)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.80s/it]


In [12]:
import os

output_dir = "/scratch/uceehuf/Mistral_orpo_acc1"
os.makedirs(output_dir, exist_ok=True)
# Check if paths exist
assert os.path.exists(output_dir), f"Output directory {output_dir} does not exist"

In [63]:
# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/scratch/uceehuf/Mistral_orpo_acc1/tokenizer_config.json',
 '/scratch/uceehuf/Mistral_orpo_acc1/special_tokens_map.json',
 '/scratch/uceehuf/Mistral_orpo_acc1/tokenizer.json')

In [13]:
#load fted model
new_model_path = output_dir
tokenizer = AutoTokenizer.from_pretrained(new_model_path)
model = AutoModelForCausalLM.from_pretrained(new_model_path,
    trust_remote_code=True, device_map="auto", torch_dtype=torch.float16)
print(f"FTed Model at path: {new_model_path} loaded")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]


FTed Model at path: /scratch/uceehuf/Mistral_orpo_acc1 loaded


In [14]:
prompt_num = len(data_test)
composed_resp = []
for i in range(1):
    model_original_resp = get_responses(prompt_num, data_test, context_full)    
    composed_resp.extend(model_original_resp)

df_fted_resp = pd.DataFrame(composed_resp, columns=['original response'])

KeyboardInterrupt: 

In [16]:

label_list_ft = extract_all_labels(data_test['Prompt'], df_fted_resp, prompt_num)

df_label_ft = pd.DataFrame(label_list_ft, columns=['Scenario', 'Text', 'Classification T1', 'Reimbursable Amount', 'Classification T2','Format Compliance'])
print("Now fine-tuned:")
evaluate(df_label_ft, data_test)

Now fine-tuned:
[2 1 0] 25
T1 acc: 0.3, T1 F1 Score: 0.3916666666666666;
T2 acc: 0.025, T2 weighted F1 Score: 0.025;
amount acc: 0.4, 
format acc: 15.000000000000002 


In [17]:
prompt_num = len(data_test)
composed_resp = []
for i in range(1):
    model_original_resp = get_responses(prompt_num, data_test, context_accom)    
    composed_resp.extend(model_original_resp)

df_fted_resp2 = pd.DataFrame(composed_resp, columns=['original response'])


the 10 th prompt responded
the 20 th prompt responded
the 30 th prompt responded
the 40 th prompt responded


In [18]:

label_list_ft2 = extract_all_labels(data_test['Prompt'], df_fted_resp2, prompt_num)

df_label_ft2 = pd.DataFrame(label_list_ft2, columns=['Scenario', 'Text', 'Classification T1', 'Reimbursable Amount', 'Classification T2','Format Compliance'])

print("Now fine-tuned, only acc policy:")
evaluate(df_label_ft2, data_test)

Now fine-tuned, only acc policy:
[2 1 0] 23
T1 acc: 0.25, T1 F1 Score: 0.36342592592592593;
T2 acc: 0.15, T2 weighted F1 Score: 0.21718227424749165;
amount acc: 0.4, 
format acc: 30.000000000000004 


In [29]:
class CustomDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer, max_prompt_length, *args, **kwargs):
        super().__init__(tokenizer, *args, **kwargs)
        self.tokenizer = tokenizer
        self.max_prompt_length = max_prompt_length

    def __call__(self, features):
        # Tokenize and truncate the prompt, chosen, and rejected
        tokenized_inputs = {
            'input_ids_prompt': [],
            'input_ids_chosen': [],
            'input_ids_rejected': [],
            'attention_mask_prompt': [],
            'attention_mask_chosen': [],
            'attention_mask_rejected': []
        }

        for feature in features:
            prompt = feature["prompt"]
            chosen = feature["chosen"]
            rejected = feature["rejected"]
            
            # Tokenize and truncate
            prompt_tokens = self.tokenizer(prompt, truncation=True, max_length=self.max_prompt_length, padding='max_length')
            chosen_tokens = self.tokenizer(chosen, truncation=True, max_length=self.max_prompt_length, padding='max_length')
            rejected_tokens = self.tokenizer(rejected, truncation=True, max_length=self.max_prompt_length, padding='max_length')

            tokenized_inputs['input_ids_prompt'].append(prompt_tokens["input_ids"])
            tokenized_inputs['input_ids_chosen'].append(chosen_tokens["input_ids"])
            tokenized_inputs['input_ids_rejected'].append(rejected_tokens["input_ids"])
            tokenized_inputs['attention_mask_prompt'].append(prompt_tokens["attention_mask"])
            tokenized_inputs['attention_mask_chosen'].append(chosen_tokens["attention_mask"])
            tokenized_inputs['attention_mask_rejected'].append(rejected_tokens["attention_mask"])
        
        # Convert lists to tensors
        for key in tokenized_inputs:
            tokenized_inputs[key] = torch.tensor(tokenized_inputs[key])

        return tokenized_inputs


In [30]:
from torch.utils.data import DataLoader
max_prompt_length = 2560  # Example value
data_collator = CustomDataCollator(tokenizer, max_prompt_length)

# Create DataLoader
dataloader = DataLoader(dataset['test'], batch_size=2, collate_fn=data_collator)

# Get a single batch
batch = next(iter(dataloader))

# Inspect tokens in the batch
for i in range(len(batch['input_ids_prompt'])):
    prompt_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_prompt'][i])
    chosen_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_chosen'][i])
    rejected_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_rejected'][i])
    
    print(f"Sample {i}:")
    print(f"  Prompt Tokens - {prompt_tokens}")
    print(f"  Chosen Tokens - {chosen_tokens}")
    print(f"  Rejected Tokens - {rejected_tokens}")




Sample 0:
  Prompt Tokens - ['<s>', '▁A', '▁<<', '<', 'POL', 'IC', 'Y', '>>>', '▁and', '▁a', '▁<<', '<', 'SC', 'EN', 'AR', 'IO', '>>>', '▁will', '▁be', '▁provided', ',', '▁you', '▁are', '▁G', 'RE', 'AT', '▁at', '▁analys', 'ing', '▁the', '▁<<', '<', 'SC', 'EN', 'AR', 'IO', '>>>', '▁according', '▁to', '▁the', '▁<<', '<', 'POL', 'IC', 'Y', '>>', '>.', '▁Your', '▁answer', '▁M', 'UST', '▁and', '▁and', '▁CAN', '▁ON', 'LY', '▁be', '▁a', '▁valid', '▁json', '▁format', ',', '▁having', '▁', '4', '▁text', '▁fields', ':', '▁Class', 'ification', '▁T', '1', ',', '▁Re', 'im', 'bur', 'se', 'ment', '▁Am', 'ount', ',', '▁Class', 'ification', '▁T', '2', ',', '▁and', '▁Re', 'asons', '.', '▁Your', '▁answer', '▁should', '▁follow', '▁these', '▁structures', ':', '▁', '<0x0A>', '▁', '1', ')', '▁"', 'Class', 'ification', '▁T', '1', '"', '▁:', 'you', '▁M', 'UST', '▁choose', '▁the', '▁activity', '▁in', '▁the', '▁<<', '<', 'sc', 'enario', '>>>', '▁is', "▁'", 'Policy', '▁Vi', 'ol', 'ated', "'", '▁or', "▁'", 'Policy'

In [10]:
torch_dtype = torch.float16

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

In [16]:
orpo_args = ORPOConfig(
    learning_rate=2e-5,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=3072,
    max_prompt_length=2560,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=5,
    #evaluation_strategy="steps",
    #eval_steps=0.05,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)


/home/uceehuf/.local/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 10/10 [00:00<00:00, 46.36 examples/s]
/home/uceehuf/.local/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [28]:
# Manually process a single batch

max_prompt_length = 2560  # Example value
data_collator = CustomDataCollator(tokenizer, max_prompt_length)
# Create DataLoader
dataloader = DataLoader(dataset['test'], batch_size=2, collate_fn=data_collator)

# Get a single batch
batch = next(iter(dataloader))

# Inspect tokens in the batch
for i in range(len(batch['input_ids_prompt'])):
    prompt_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_prompt'][i])
    chosen_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_chosen'][i])
    rejected_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids_rejected'][i])
    
    print(f"Sample {i}:")
    print(f"  Prompt Tokens - {prompt_tokens}")
    print(f"  Chosen Tokens - {chosen_tokens}")
    print(f"  Rejected Tokens - {rejected_tokens}")


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['prompt', 'chosen', 'rejected', 'input_ids_prompt', 'input_ids_chosen', 'input_ids_rejected']

In [12]:

# Start training
orpo_trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junyu-zhu-23 (ucl_student). Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 166.00 MiB. GPU 0 has a total capacity of 79.14 GiB of which 118.56 MiB is free. Process 1870000 has 414.00 MiB memory in use. Process 1877952 has 414.00 MiB memory in use. Process 1881595 has 414.00 MiB memory in use. Including non-PyTorch memory, this process has 77.77 GiB memory in use. Of the allocated memory 72.36 GiB is allocated by PyTorch, and 4.92 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

--- Logging error ---
Traceback (most recent call last):
  File "/home/uceehuf/.local/lib/python3.11/site-packages/wandb/sdk/lib/sock_client.py", line 255, in _read_packet_bytes
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/uceehuf/.local/lib/python3.11/site-packages/wandb/sdk/interface/router_sock.py", line 27, in _read_message
  File "/home/uceehuf/.local/lib/python3.11/site-packages/wandb/sdk/lib/sock_client.py", line 285, in read_server_response
  File "/home/uceehuf/.local/lib/python3.11/site-packages/wandb/sdk/lib/sock_client.py", line 259, in _read_packet_bytes
wandb.sdk.lib.sock_client.SockClientClosedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/uceehuf/.local/lib/python3.11/site-packages/wandb/sdk/interface/router.py", line 70, in message_loop
  File "/home/uceehuf/.lo